In [9]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [2]:
import pandas as pd
import numpy as np

In [4]:
import xgboost as xgb

In [10]:
from utils import UtilsKy

In [14]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

In [11]:
def rmsle(predt: np.ndarray, dtrain: xgb.DMatrix):
    ''' Root mean squared log error metric.'''
    y = dtrain.get_label()
    print("min ={}, max ={}". format(min(predt), max(predt)))
    predt[predt < -1] = -1 + 1e-6
    #predt = np.where(predt > 0.25, y, predt)
    elements = np.power(np.log1p(y) - np.log1p(predt), 2)
    print("min ={}, max ={}". format(min(elements), max(elements)))
    
    return 'PyRMSLE', float(np.sqrt(np.sum(elements) / len(y)))

In [12]:
from typing import Tuple
def gradient(predt: np.ndarray, dtrain: xgb.DMatrix) -> np.ndarray:
    '''Compute the gradient squared log error.'''
    y = dtrain.get_label()
    return (np.log1p(predt) - np.log1p(y)) / (predt + 1)

def hessian(predt: np.ndarray, dtrain: xgb.DMatrix) -> np.ndarray:
    '''Compute the hessian for squared log error.'''
    y = dtrain.get_label()
    return ((-np.log1p(predt) + np.log1p(y) + 1) /
            np.power(predt + 1, 2))

def squared_log(predt: np.ndarray,
                dtrain: xgb.DMatrix) -> Tuple[np.ndarray, np.ndarray]:
    '''Squared Log Error objective. A simplified version for RMSLE used as
    objective function.
    '''
    predt[predt < -1] = -1 + 1e-6
#     print('==========================================')
    grad = gradient(predt, dtrain)
    hess = hessian(predt, dtrain)    
    return grad, hess

In [13]:
# kyw3
path_data = '/mnt/files/workdata/work/merchants/merchant_33_kyw3_2020-06-05/04_experiments/ex_01_some_teach/'
db_teach = pd.read_csv(UtilsKy.DB_TEACH_KYW3, dtype=str, encoding='cp1251')
db_test = pd.read_csv(UtilsKy.DB_TEST_KYW3, dtype=str, encoding='cp1251')

In [15]:
COL_FACTORS = ['bin', 'amount', 'bank_currency', 'hour', 'day_of_week', 'longitude', 'latitude', 'phone_2_norm', 'is_gender_undefined', 'is_city_resolved']

In [16]:
train = db_teach[COL_FACTORS]
test = db_test[COL_FACTORS]

In [17]:
train = train.apply(pd.to_numeric, errors="coerce")
test = test.apply(pd.to_numeric, errors="coerce")

In [18]:
replace_val = -9999
train = train.fillna(replace_val)
test = test.fillna(replace_val)
replace_val

-9999

In [19]:
train = train.values
test = test.values
label = db_teach.status

In [20]:
dtrain = xgb.DMatrix(train, label)
dtest = xgb.DMatrix(test)

In [21]:
param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'

In [22]:
bst = xgb.train(param, dtrain=dtrain, num_boost_round=10, verbose_eval=3, feval=rmsle)

In [23]:
bst.predict(dtest)

array([0.01707395, 0.02037124, 0.00598064, ..., 0.00997019, 0.00997019,
       0.02628254], dtype=float32)